
# 🇴🇲 Oman Trade Products — Plotly (Step‑by‑Step, Imports Excluded)

This notebook is split into small, independent steps so you can run or re‑run parts separately.
**Totals exclude Imports** (Total Exports = Export + Re‑Export). Non‑Oil & Oil use **Export only**.


## 1) Environment & Renderer

In [75]:

import plotly.io as pio

try:
    import nbformat, IPython, ipywidgets
    from packaging.version import Version
    assert Version(nbformat.__version__) >= Version("4.2.0")
    pio.renderers.default = "vscode"  # Inline in VS Code
    print("✅ Plotly renderer:", pio.renderers.default)
except Exception as e:
    print(f"⚠️ Inline renderer unavailable ({e}) — using browser fallback.")
    pio.renderers.default = "browser"


✅ Plotly renderer: vscode


## 2) Imports

In [76]:

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

pd.set_option("display.float_format", lambda x: f"{x:,.0f}")


## 3) Paths

In [77]:

CSV_PATH = Path("/Users/Yasser/Downloads/oman_trade_products.csv")
print("Data path:", CSV_PATH)


Data path: /Users/Yasser/Downloads/oman_trade_products.csv


## 4) Load Data

In [78]:

df = pd.read_csv(CSV_PATH)
print("Rows:", len(df))
df.head(10)


Rows: 249883


,year,trade_type,hs_code,hs_name,value_omr,date,hs_code_full,hs_full_int,section_code_2,section_name,section_name_clean,bec_broad
0,1998,Export,"1,012,110",Purebred Horses Of Arab breed,"20,300",1998,10121100,10121100,2,VEGETABLE PRODUCTS,Vegetable products,Food & Beverages
1,1998,Export,"1,019,000",Live Mules And Hinnies,7,1998,10190000,10190000,2,VEGETABLE PRODUCTS,Vegetable products,Food & Beverages
2,1998,Export,"1,022,100",PureBred Cattle For Breeding,"19,350",1998,10221000,10221000,2,VEGETABLE PRODUCTS,Vegetable products,Food & Beverages
3,1998,Export,"1,029,000",Live Bovine Animals Excl Cattle And Buffalo,"16,970",1998,10290000,10290000,2,VEGETABLE PRODUCTS,Vegetable products,Food & Beverages
4,1998,Export,"1,042,010",Live PureBred Breeding Goats,"1,220,112",1998,10420100,10420100,2,VEGETABLE PRODUCTS,Vegetable products,Food & Beverages
5,1998,Export,"1,051,100",Live Fowls Of The Species Gallus Domesticus We...,"277,280",1998,10511000,10511000,2,VEGETABLE PRODUCTS,Vegetable products,Food & Beverages
6,1998,Export,"1,059,990",Live Ducks Geese Excluding Tame Weighing More ...,"45,919",1998,10599900,10599900,2,VEGETABLE PRODUCTS,Vegetable products,Food & Beverages
7,1998,Export,"1,061,100",Live Primates,"4,642,045",1998,10611000,10611000,2,VEGETABLE PRODUCTS,Vegetable products,Food & Beverages
8,1998,Export,"2,023,090",Meat Of Bovine Animals Excluding Minced Meat B...,"433,296",1998,20230900,20230900,4,PREPARED FOODSTUFFS,Prepared foods; beverages; tobacco,Consumer Goods
9,1998,Export,"2,045,011",Carcasses HalfCarcasses Of Goats Meat Fresh Or...,"2,450",1998,20450110,20450110,4,PREPARED FOODSTUFFS,Prepared foods; beverages; tobacco,Consumer Goods


## 5) Clean Columns & Types

In [79]:

df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")
df["trade_type"] = df["trade_type"].astype(str).str.title()
df["hs_code"] = df["hs_code"].astype(str).str.strip()
df["value_omr"] = pd.to_numeric(df["value_omr"], errors="coerce").fillna(0.0)
print("✅ Cleaned. Columns:", list(df.columns))
df.sample(5)


✅ Cleaned. Columns: ['year', 'trade_type', 'hs_code', 'hs_name', 'value_omr', 'date', 'hs_code_full', 'hs_full_int', 'section_code_2', 'section_name', 'section_name_clean', 'bec_broad']


,year,trade_type,hs_code,hs_name,value_omr,date,hs_code_full,hs_full_int,section_code_2,section_name,section_name_clean,bec_broad
28668,2003,Import,33030090.0,Perfumes Toilet Waters NES,"2,640,930",2003,33030090,33030090,6,PRODUCTS OF THE CHEMICALS,Chemical products,Industrial Supplies
244160,2024,Import,84383000.0,Machinery For Sugar Manufacture Excl Centrifug...,"3,211",2024,84383000,84383000,16,MACHINERY AND MECHANICAL APPLIANCES,"Machinery, electrical equipment & parts",Capital Goods
9335,2000,Import,42021220.0,BriefCases With Outer Surface Of Plastic Or Of...,"48,251",2000,42021220,42021220,8,"RAW HIDES AND SKINS, FURKINS AND","Hides, leather, furskins & articles",Consumer Goods
141667,2016,Import,28309000.0,Sulphides Excl Sodium Polysulphides Whether Or...,7,2016,28309000,28309000,6,PRODUCTS OF THE CHEMICALS,Chemical products,Industrial Supplies
129628,2015,Import,3056900.0,Fish Salted Or In Brine Only Excl Fillets Offa...,"10,278",2015,30569000,30569000,6,PRODUCTS OF THE CHEMICALS,Chemical products,Industrial Supplies


## 6) Oil vs Non‑Oil flag (HS 27 = Oil)

In [80]:

df["oil_flag"] = np.where(df["hs_code"].str.startswith("27"), "Oil (HS 27)", "Non‑Oil")
df["oil_flag"].value_counts()


oil_flag
Non‑Oil        246939
Oil (HS 27)      2944
Name: count, dtype: int64

## 7) Totals (Exclude Imports)

In [81]:

# Total Exports = Export + Re‑Export
exports_plus_re = df[df["trade_type"].isin(["Export","Re-Export"])].copy()
total = (exports_plus_re.groupby("year", as_index=False)["value_omr"]
         .sum().rename(columns={"value_omr":"Total_Exports_All"}))
total.head()


,year,Total_Exports_All
0,1998,"2,071,803,925"
1,1999,"2,727,083,011"
2,2000,"4,281,139,167"
3,2001,"4,229,515,282"
4,2002,"4,257,236,204"


## 8) Non‑Oil and Oil (Export only)

In [82]:

non_oil = (df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Non‑Oil")]
           .groupby("year", as_index=False)["value_omr"].sum()
           .rename(columns={"value_omr":"NonOil_Exports"}))

oil = (df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Oil (HS 27)")]
       .groupby("year", as_index=False)["value_omr"].sum()
       .rename(columns={"value_omr":"Oil_Exports"}))

non_oil.head(), oil.head()


(   year  NonOil_Exports
 0  1998     186,394,040
 1  1999     187,295,006
 2  2000     233,000,980
 3  2001     251,264,287
 4  2002     253,496,451,
    year   Oil_Exports
 0  1998 1,392,154,980
 1  1999 2,084,443,561
 2  2000 3,549,505,207
 3  2001 3,400,639,684
 4  2002 3,277,036,168)

## 9) Merge Series & Compute Share/YoY

In [83]:

series = (total.merge(non_oil, on="year", how="outer")
               .merge(oil, on="year", how="outer")
               .fillna(0.0)
               .sort_values("year"))

series["NonOil_Share_pct"] = np.where(series["Total_Exports_All"]>0,
                                      (series["NonOil_Exports"]/series["Total_Exports_All"])*100, np.nan)
series["YoY_Total_%"]  = series["Total_Exports_All"].pct_change()*100
series["YoY_NonOil_%"] = series["NonOil_Exports"].pct_change()*100
series["YoY_Oil_%"]    = series["Oil_Exports"].pct_change()*100
series.head(10)


,year,Total_Exports_All,NonOil_Exports,Oil_Exports,NonOil_Share_pct,YoY_Total_%,YoY_NonOil_%,YoY_Oil_%
0,1998,"2,071,803,925","186,394,040","1,392,154,980",9,NaN,NaN,NaN
1,1999,"2,727,083,011","187,295,006","2,084,443,561",7,32,0,50
2,2000,"4,281,139,167","233,000,980","3,549,505,207",5,57,24,70
3,2001,"4,229,515,282","251,264,287","3,400,639,684",6,-1,8,-4
4,2002,"4,257,236,204","253,496,451","3,277,036,168",6,1,1,-4
5,2003,"4,425,279,368","298,931,539","3,525,578,278",7,4,18,8
6,2004,"5,083,561,968","407,065,064","4,138,208,017",8,15,36,17
7,2005,"7,114,460,435","539,218,830","5,991,334,390",8,40,32,45
8,2006,"8,370,404,461","648,567,870","6,954,891,285",8,18,20,16
9,2007,"9,157,238,607","912,965,640","7,240,778,668",10,9,41,4


## 10) Chart — Total vs Non‑Oil Exports

In [84]:

fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=series["year"], y=series["Total_Exports_All"],
                          mode="lines+markers", name="Total Exports (Export + Re‑Export)",
                          line=dict(width=3)))
fig1.add_trace(go.Scatter(x=series["year"], y=series["NonOil_Exports"],
                          mode="lines+markers", name="Non‑Oil Exports (Export only)",
                          line=dict(width=3)))
fig1.update_layout(title="Oman: Total vs Non‑Oil Exports (Levels) — Imports Excluded from Total",
                   xaxis_title="Year", yaxis_title="OMR", template="plotly_white")
fig1.show()


## 11) Chart — Non‑Oil Share of Total Exports

In [85]:

fig2 = px.line(series, x="year", y="NonOil_Share_pct",
               title="Non‑Oil Exports as % of Total Exports (Export + Re‑Export)",
               markers=True)
fig2.update_traces(line=dict(width=3), marker=dict(size=8))
fig2.update_layout(template="plotly_white", yaxis_title="Share (%)")
fig2.show()


## 12) Chart — YoY Growth

In [86]:

fig3 = go.Figure()
for col, label in [("YoY_Total_%","Total Exports"), ("YoY_NonOil_%","Non‑Oil Exports"), ("YoY_Oil_%","Oil Exports")]:
    fig3.add_trace(go.Scatter(x=series["year"], y=series[col], mode="lines+markers", name=label))
fig3.add_hline(y=0, line_dash="dot", line_color="gray")
fig3.update_layout(title="YoY Growth: Total, Non‑Oil, and Oil Exports",
                   xaxis_title="Year", yaxis_title="YoY Growth (%)", template="plotly_white")
fig3.show()


## 13) Chart — Number of Exported Products (Export only)

In [87]:

exp_only = df[df["trade_type"]=="Export"]
counts = (exp_only[exp_only["value_omr"]>0]
          .groupby(["year","oil_flag"])["hs_code"].nunique()
          .reset_index())

fig4 = px.line(counts, x="year", y="hs_code", color="oil_flag",
               markers=True, title="Number of Products Exported (Export only)")
fig4.update_traces(line=dict(width=3))
fig4.update_layout(template="plotly_white", yaxis_title="Unique HS Codes")
fig4.show()


## 14) Build BEC Aggregates

In [88]:

# Non‑Oil Exports by BEC (Export only)
non_oil_bec = (df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Non‑Oil")]
               .groupby(["year","bec_broad"], as_index=False)["value_omr"].sum())

# Total Exports by BEC (Export + Re‑Export; Imports excluded)
total_bec = (df[df["trade_type"].isin(["Export","Re-Export"])]
             .groupby(["year","bec_broad"], as_index=False)["value_omr"].sum())

non_oil_bec.head(), total_bec.head()


(   year            bec_broad  value_omr
 0  1998        Capital Goods 15,478,039
 1  1998       Consumer Goods 56,912,438
 2  1998     Food & Beverages 18,567,947
 3  1998   Fuels & Lubricants  5,253,125
 4  1998  Industrial Supplies 89,898,845,
    year            bec_broad     value_omr
 0  1998        Capital Goods   115,292,346
 1  1998       Consumer Goods   151,677,852
 2  1998     Food & Beverages    28,914,844
 3  1998   Fuels & Lubricants 1,399,275,995
 4  1998  Industrial Supplies   129,795,343)

## 15) Charts — BEC (Stacked Areas)

In [89]:

def plot_stacked(df_bec, title):
    wide = df_bec.pivot(index="year", columns="bec_broad", values="value_omr").fillna(0).sort_index()
    fig = go.Figure()
    for col in wide.columns:
        fig.add_trace(go.Scatter(x=wide.index, y=wide[col], stackgroup="one", name=col))
    fig.update_layout(title=title, xaxis_title="Year", yaxis_title="OMR", template="plotly_white")
    fig.show()

plot_stacked(non_oil_bec, "Non‑Oil Exports by BEC (Export only)")
plot_stacked(total_bec, "Total Exports by BEC (Export + Re‑Export; Imports EXCLUDED)")


## 16) Build HS Sections (Non‑Oil, Export only)

In [90]:

non_oil_section = (df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Non‑Oil")]
                   .groupby(["year","section_name_clean"], as_index=False)["value_omr"].sum())

tot_by_year = (non_oil_section.groupby("year", as_index=False)["value_omr"]
               .sum().rename(columns={"value_omr":"total"}))
sec_share = non_oil_section.merge(tot_by_year, on="year", how="left")
sec_share["share"] = np.where(sec_share["total"]>0, sec_share["value_omr"]/sec_share["total"], 0.0)

top_sections = (sec_share.groupby("section_name_clean")["share"]
                .mean().sort_values(ascending=False).head(8).index.tolist())

non_oil_section["section_grouped"] = non_oil_section["section_name_clean"].where(
    non_oil_section["section_name_clean"].isin(top_sections), "Other"
)
non_oil_section.head(10)


,year,section_name_clean,value_omr,section_grouped
0,1998,Animal/vegetable fats & oils,"9,023,304",Other
1,1998,Arms & ammunition,"15,432",Other
2,1998,Base metals & articles,"31,400,770",Base metals & articles
3,1998,Chemical products,"31,831,806",Chemical products
4,1998,"Footwear, headgear, umbrellas","358,451",Other
5,1998,"Hides, leather, furskins & articles","79,495",Other
6,1998,"Machinery, electrical equipment & parts","15,126,630","Machinery, electrical equipment & parts"
7,1998,Mineral products,"5,253,125",Mineral products
8,1998,Miscellaneous manufactured articles,"12,217,798",Other
9,1998,"Pearls, precious stones & metals; jewellery; coin","647,372",Other


## 17) Chart — Non‑Oil Exports by HS Section (Top 8 + Other)

In [91]:

wide = (non_oil_section.pivot(index="year", columns="section_grouped", values="value_omr")
        .fillna(0).sort_index())

fig7 = go.Figure()
for col in wide.columns:
    fig7.add_trace(go.Scatter(x=wide.index, y=wide[col], stackgroup="one", name=col))
fig7.update_layout(title="Non‑Oil Exports by HS Section — Top 8 + Other (Export only)",
                   xaxis_title="Year", yaxis_title="OMR", template="plotly_white")
fig7.show()


ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
print('Please open the v2 file; this is a placeholder if the base is missing.')

## 18) Contribution to **Total Exports** Growth (Oil / Non‑Oil / Re‑Export)

In [ ]:

exports_plus_re = df[df["trade_type"].isin(["Export","Re-Export"])].copy()

oil_exp = (df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Oil (HS 27)")]
           .groupby("year", as_index=False)["value_omr"].sum()
           .rename(columns={"value_omr":"oil_export"}))

non_oil_exp = (df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Non‑Oil")]
               .groupby("year", as_index=False)["value_omr"].sum()
               .rename(columns={"value_omr":"non_oil_export"}))

reexp = (df[df["trade_type"]=="Re-Export"]
         .groupby("year", as_index=False)["value_omr"].sum()
         .rename(columns={"value_omr":"reexport"}))

totals = (exports_plus_re.groupby("year", as_index=False)["value_omr"].sum()
          .rename(columns={"value_omr":"total_exports"}))

comp = (totals.merge(oil_exp, on="year", how="left")
               .merge(non_oil_exp, on="year", how="left")
               .merge(reexp, on="year", how="left")).fillna(0.0).sort_values("year")

comp["total_prev"] = comp["total_exports"].shift(1)
for col in ["oil_export","non_oil_export","reexport","total_exports"]:
    comp[f"delta_{col}"] = comp[col].diff()

comp["contrib_pp_oil"]      = np.where(comp["total_prev"]>0, comp["delta_oil_export"]     / comp["total_prev"] * 100, np.nan)
comp["contrib_pp_non_oil"]  = np.where(comp["total_prev"]>0, comp["delta_non_oil_export"] / comp["total_prev"] * 100, np.nan)
comp["contrib_pp_reexport"] = np.where(comp["total_prev"]>0, comp["delta_reexport"]       / comp["total_prev"] * 100, np.nan)
comp["yoy_total_%"]         = np.where(comp["total_prev"]>0, comp["delta_total_exports"]  / comp["total_prev"] * 100, np.nan)

comp[["year","total_exports","yoy_total_%","contrib_pp_oil","contrib_pp_non_oil","contrib_pp_reexport"]].head(10)


,year,total_exports,yoy_total_%,contrib_pp_oil,contrib_pp_non_oil,contrib_pp_reexport
0,1998,"2,071,803,925",NaN,NaN,NaN,NaN
1,1999,"2,727,083,011",32,33,0,-2
2,2000,"4,281,139,167",57,54,2,2
3,2001,"4,229,515,282",-1,-3,0,2
4,2002,"4,257,236,204",1,-3,0,4
5,2003,"4,425,279,368",4,6,1,-3
6,2004,"5,083,561,968",15,14,2,-1
7,2005,"7,114,460,435",40,36,3,1
8,2006,"8,370,404,461",18,14,2,3
9,2007,"9,157,238,607",9,3,3,3


In [ ]:

import plotly.graph_objects as go
fig18 = go.Figure()
fig18.add_bar(x=comp["year"], y=comp["contrib_pp_non_oil"], name="Non‑Oil (pp)")
fig18.add_bar(x=comp["year"], y=comp["contrib_pp_oil"], name="Oil (pp)")
fig18.add_bar(x=comp["year"], y=comp["contrib_pp_reexport"], name="Re‑Export (pp)")
fig18.update_layout(barmode="relative", title="Contribution to Total Exports YoY Growth (pp) — Imports Excluded from Total",
                    xaxis_title="Year", yaxis_title="Percentage points", template="plotly_white")
fig18.show()


## 19) Contribution to **Non‑Oil Exports** Growth by BEC

In [ ]:

no_bec = (df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Non‑Oil")]
          .groupby(["year","bec_broad"], as_index=False)["value_omr"].sum())

no_total = (df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Non‑Oil")]
            .groupby("year", as_index=False)["value_omr"].sum()
            .rename(columns={"value_omr":"non_oil_total"}))

wide = no_bec.pivot(index="year", columns="bec_broad", values="value_omr").fillna(0.0).sort_index()
wide = wide.merge(no_total.set_index("year"), left_index=True, right_index=True, how="left")

wide["total_prev"] = wide["non_oil_total"].shift(1)
deltas = wide.drop(columns=["non_oil_total","total_prev"]).diff()
contrib_pp = deltas.div(wide["total_prev"], axis=0) * 100
contrib_pp = contrib_pp.reset_index().melt(id_vars="year", var_name="BEC", value_name="contrib_pp")

contrib_pp.head(10)


,year,BEC,contrib_pp
0,1998,Capital Goods,NaN
1,1999,Capital Goods,-3
2,2000,Capital Goods,2
3,2001,Capital Goods,2
4,2002,Capital Goods,1
5,2003,Capital Goods,1
6,2004,Capital Goods,3
7,2005,Capital Goods,4
8,2006,Capital Goods,8
9,2007,Capital Goods,14


In [ ]:

import plotly.express as px
fig19 = px.bar(contrib_pp, x="year", y="contrib_pp", color="BEC",
               title="Contribution to Non‑Oil Exports YoY Growth by BEC (pp)",
               labels={"contrib_pp":"Percentage points"})
fig19.update_layout(barmode="relative", template="plotly_white")
fig19.show()


In [ ]:

last_years = contrib_pp["year"].dropna().unique()
last_years = np.sort(last_years)[-5:] if len(last_years)>5 else np.sort(last_years)

tbl = []
for y in last_years:
    sub = contrib_pp[contrib_pp["year"]==y].dropna(subset=["contrib_pp"])
    if len(sub)==0: 
        continue
    top_pos = sub.nlargest(3, "contrib_pp")
    top_neg = sub.nsmallest(3, "contrib_pp")
    for rank, row in enumerate(top_pos.itertuples(index=False), 1):
        tbl.append({"year": int(y), "type":"Top +", "rank": rank, "BEC": row.BEC, "pp": row.contrib_pp})
    for rank, row in enumerate(top_neg.itertuples(index=False), 1):
        tbl.append({"year": int(y), "type":"Top -", "rank": rank, "BEC": row.BEC, "pp": row.contrib_pp})

import pandas as pd
tbl_df = pd.DataFrame(tbl)
tbl_df


,year,type,rank,BEC,pp
0,2020,Top +,1,Industrial Supplies,8
1,2020,Top +,2,Fuels & Lubricants,2
2,2020,Top +,3,Consumer Goods,2
3,2020,Top -,1,Transport Equipment,-5
4,2020,Top -,2,Special Category,-0
5,2020,Top -,3,Food & Beverages,-0
6,2021,Top +,1,Industrial Supplies,41
7,2021,Top +,2,Fuels & Lubricants,4
8,2021,Top +,3,Transport Equipment,1
9,2021,Top -,1,Consumer Goods,-1


## 20) Top **Non‑Oil** Products (value & growth)

In [ ]:

latest_year = int(df["year"].dropna().max())
prev_year = latest_year - 1
print("Latest year:", latest_year, "| Previous:", prev_year)

non_oil_export = df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Non‑Oil")].copy()

cur = (non_oil_export[non_oil_export["year"]==latest_year]
       .groupby(["hs_code","hs_name"], as_index=False)["value_omr"].sum()
       .rename(columns={"value_omr":"value_curr"}))

prev = (non_oil_export[non_oil_export["year"]==prev_year]
        .groupby(["hs_code","hs_name"], as_index=False)["value_omr"].sum()
        .rename(columns={"value_omr":"value_prev"}))

prod = cur.merge(prev, on=["hs_code","hs_name"], how="left").fillna({"value_prev":0.0})
prod["abs_delta"] = prod["value_curr"] - prod["value_prev"]
prod["yoy_pct"] = np.where(prod["value_prev"]>0, (prod["abs_delta"]/prod["value_prev"])*100, np.nan)

top_val = prod.sort_values("value_curr", ascending=False).head(10)
top_grow = prod.sort_values("abs_delta", ascending=False).head(10)

print("Top 10 by value:")
top_val[["hs_code","hs_name","value_curr","value_prev","abs_delta","yoy_pct"]]


Latest year: 2024 | Previous: 2023
Top 10 by value:


,hs_code,hs_name,value_curr,value_prev,abs_delta,yoy_pct
273,26011200.0,Agglomerated Iron Ores And Concentrates Excl R...,"674,724,865","635,262,553","39,462,312",6
345,31021000.0,Urea Whether Or Not In Aqueous Solution Excl T...,"421,099,044","418,104,969","2,994,075",1
1029,72142040.0,Other bars and rods of iron or nonalloy steel ...,"249,612,049","223,470,552","26,141,497",12
308,29051100.0,Methanol Methyl Alcohol,"236,365,241","198,210,177","38,155,064",19
1193,76011000.0,Aluminium Not Alloyed Unwrought,"222,067,770","249,100,878","-27,033,108",-11
501,39076100.0,poly ethylene terephthalate Having a viscosity...,"204,902,697","161,365,837","43,536,860",27
535,39206200.0,Plates Sheets Film Foil And Strip Of NonCellul...,"172,083,069","157,979,433","14,103,636",9
999,72071100.0,SemiFinished Products Of Iron Or NonAlloy Stee...,"167,791,621","120,271,006","47,520,615",40
488,39021000.0,Polypropylene In Primary Forms,"163,900,363","174,344,707","-10,444,344",-6
484,39012000.0,Polyethylene With A Specific Gravity Of 094 In...,"149,311,837","102,391,061","46,920,776",46


In [ ]:

print("Top 10 by growth (absolute):")
top_grow[["hs_code","hs_name","value_curr","value_prev","abs_delta","yoy_pct"]]


Top 10 by growth (absolute):


,hs_code,hs_name,value_curr,value_prev,abs_delta,yoy_pct
999,72071100.0,SemiFinished Products Of Iron Or NonAlloy Stee...,"167,791,621","120,271,006","47,520,615",40
484,39012000.0,Polyethylene With A Specific Gravity Of 094 In...,"149,311,837","102,391,061","46,920,776",46
501,39076100.0,poly ethylene terephthalate Having a viscosity...,"204,902,697","161,365,837","43,536,860",27
273,26011200.0,Agglomerated Iron Ores And Concentrates Excl R...,"674,724,865","635,262,553","39,462,312",6
308,29051100.0,Methanol Methyl Alcohol,"236,365,241","198,210,177","38,155,064",19
489,39023000.0,Propylene Copolymers In Primary Forms,"30,887,279","2,345,724","28,541,555","1,217"
1029,72142040.0,Other bars and rods of iron or nonalloy steel ...,"249,612,049","223,470,552","26,141,497",12
1206,76061230.0,Aluminium flattened or grained coils and strip...,"99,060,623","74,190,680","24,869,943",34
486,39014000.0,Ethylenealphaolefin copolymers having a specif...,"40,298,805","18,156,839","22,141,966",122
976,71081210.0,Ingots Of Gold NonMonetary In Unwrought Forms,"40,185,085","23,410,651","16,774,434",72


In [ ]:

import plotly.express as px
fig20 = px.bar(top_val.sort_values("value_curr"), 
               x="value_curr", y="hs_name", orientation="h",
               title=f"Top 10 Non‑Oil Products by Value — {latest_year}",
               labels={"value_curr":"OMR","hs_name":""})
fig20.update_layout(template="plotly_white")
fig20.show()


## 21) HS Sections — Growth & Share over Time

In [ ]:

sec = (df[(df["trade_type"]=="Export")]
       .groupby(["year","section_name_clean","oil_flag"], as_index=False)["value_omr"].sum())

sec_no = sec[sec["oil_flag"]=="Non‑Oil"].drop(columns=["oil_flag"])
wide_sec = sec_no.pivot(index="year", columns="section_name_clean", values="value_omr").fillna(0.0).sort_index()

growth_sec = wide_sec.pct_change()*100
growth_sec = growth_sec.replace([np.inf,-np.inf], np.nan)

year_tot_no = wide_sec.sum(axis=1).replace(0, np.nan)
share_sec = wide_sec.div(year_tot_no, axis=0)*100

print("Non‑Oil section totals (wide):")
wide_sec.tail(3)


Non‑Oil section totals (wide):


section_name_clean,Animal/vegetable fats & oils,Arms & ammunition,Base metals & articles,Chemical products,"Footwear, headgear, umbrellas","Hides, leather, furskins & articles","Machinery, electrical equipment & parts",Mineral products,Miscellaneous manufactured articles,"Pearls, precious stones & metals; jewellery; coin",Plastics & rubber,Precision/medical instruments; clocks; musical,Prepared foods; beverages; tobacco,"Pulp, paper & paperboard; articles","Stone, plaster, cement; ceramics; glass",Textiles & textile articles,Vegetable products,"Vehicles, aircraft & vessels","Wood, cork & basketwork",Works of art & antiques
year,,,,,,,,,,,,,,,,,,,,
2022,"121,496,413",0,"1,483,447,104","1,596,383,951","833,303","359,137","157,596,853","1,060,131,388","13,504,789","13,430,763","1,271,428,360","6,908,617","230,283,566","48,055,302","145,381,488","21,323,443","59,075,644","16,590,155","1,839,855","4,329,937"
2023,"102,495,127",0,"1,328,080,603","1,208,960,417","971,022","371,616","369,872,329","849,108,909","12,955,500","40,384,094","1,029,691,798","3,583,250","243,257,834","65,186,812","126,205,704","9,642,635","53,521,562","22,305,005","1,550,648","4,040,040"
2024,"102,363,966","188,650","1,319,787,074","804,172,153","1,182,175","433,435","271,741,252","872,171,869","15,315,685","75,584,472","995,652,716","2,681,422","235,768,647","55,751,560","92,020,083","8,478,829","55,505,315","28,762,622","1,891,166","3,113,348"


In [ ]:

print("YoY growth (%):")
growth_sec.tail(3)


YoY growth (%):


section_name_clean,Animal/vegetable fats & oils,Arms & ammunition,Base metals & articles,Chemical products,"Footwear, headgear, umbrellas","Hides, leather, furskins & articles","Machinery, electrical equipment & parts",Mineral products,Miscellaneous manufactured articles,"Pearls, precious stones & metals; jewellery; coin",Plastics & rubber,Precision/medical instruments; clocks; musical,Prepared foods; beverages; tobacco,"Pulp, paper & paperboard; articles","Stone, plaster, cement; ceramics; glass",Textiles & textile articles,Vegetable products,"Vehicles, aircraft & vessels","Wood, cork & basketwork",Works of art & antiques
year,,,,,,,,,,,,,,,,,,,,
2022,33,NaN,13,20,-25,-5,1,95,-3,-10,29,13,13,-38,31,-16,16,-79,15,-11
2023,-16,NaN,-10,-24,17,3,135,-20,-4,201,-19,-48,6,36,-13,-55,-9,34,-16,-7
2024,-0,NaN,-1,-33,22,17,-27,3,18,87,-3,-25,-3,-14,-27,-12,4,29,22,-23


In [ ]:

print("Share of non‑oil (%):")
share_sec.tail(3)


In [ ]:

import plotly.express as px
fig21 = px.imshow(growth_sec.T, aspect='auto', color_continuous_scale='RdBu', origin='lower',
                  title='YoY Growth (%) — Non‑Oil Exports by HS Section',
                  labels=dict(x="Year", y="HS Section", color="% YoY"))
fig21.update_layout(template="plotly_white")
fig21.show()


## 22) Contribution to **Non‑Oil** Growth by HS Section (pp)

In [ ]:

no_sec = (df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Non‑Oil")]
          .groupby(["year","section_name_clean"], as_index=False)["value_omr"].sum())

no_tot = (df[(df["trade_type"]=="Export") & (df["oil_flag"]=="Non‑Oil")]
          .groupby("year", as_index=False)["value_omr"].sum().rename(columns={"value_omr":"non_oil_total"}))

pivot_sec = no_sec.pivot(index="year", columns="section_name_clean", values="value_omr").fillna(0.0).sort_index()
pivot_sec = pivot_sec.merge(no_tot.set_index("year"), left_index=True, right_index=True, how="left")

pivot_sec["total_prev"] = pivot_sec["non_oil_total"].shift(1)
deltas = pivot_sec.drop(columns=["non_oil_total","total_prev"]).diff()
contrib_pp_sec = deltas.div(pivot_sec["total_prev"], axis=0) * 100

contrib_long = contrib_pp_sec.reset_index().melt(id_vars="year", var_name="HS Section", value_name="pp")
contrib_long.head(10)


,year,HS Section,pp
0,1998,Animal/vegetable fats & oils,NaN
1,1999,Animal/vegetable fats & oils,-0
2,2000,Animal/vegetable fats & oils,1
3,2001,Animal/vegetable fats & oils,-1
4,2002,Animal/vegetable fats & oils,0
5,2003,Animal/vegetable fats & oils,2
6,2004,Animal/vegetable fats & oils,2
7,2005,Animal/vegetable fats & oils,3
8,2006,Animal/vegetable fats & oils,0
9,2007,Animal/vegetable fats & oils,1


In [ ]:

import plotly.express as px
fig22 = px.bar(contrib_long, x="year", y="pp", color="HS Section",
               title="Contribution to Non‑Oil Exports YoY Growth by HS Section (pp)",
               labels={"pp":"Percentage points"})
fig22.update_layout(barmode="relative", template="plotly_white")
fig22.show()
